# Project - Airline AI Assistant

We'll now bring together what we've learned to make an AI Customer Support assistant for an Airline

In [1]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [2]:
# Initialization

load_dotenv()

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4o-mini"
openai = OpenAI()

OpenAI API Key exists and begins sk-proj-


In [3]:
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."
system_message += "Do not make up answers."

In [4]:
# This function looks rather simpler than the one from my video, because we're taking advantage of the latest Gradio updates

def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages)
    return response.choices[0].message.content

gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


## Tools

Tools are an incredibly powerful feature provided by the frontier LLMs.

With tools, you can write a function, and have the LLM call that function as part of its response.

Sounds almost spooky.. we're giving it the power to run code on our machine?

Well, kinda.

In [5]:
import sqlite3
def get_ticket_price(destination_city, ticket_class):
    print(f"Tool get_ticket_price called for {destination_city}, {ticket_class}")
    conn = sqlite3.connect("prices.db")
    cursor = conn.cursor()
    query = f"select prices.{ticket_class.lower()} from prices where lower(prices.Destination) = '{destination_city.lower()}'"
    print (f'{query=}')
    try:
        cursor.execute(query)
        results = cursor.fetchall()
        if results:
            return f"${results[0][0]}"
        else:
            return "Unknown"
    except sqlite3.Error as e:
        print("Error:", e)
    finally:
        cursor.close()
        conn.close()

In [6]:
get_ticket_price("Rome", "business")

Tool get_ticket_price called for Rome, business
query="select prices.business from prices where lower(prices.Destination) = 'rome'"


'$1050'

In [7]:
# There's a particular dictionary structure that's required to describe our function:

price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city for the given ticket class. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a First Class ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
            "ticket_class": {
                "type": "string",
                "description": "The ticket class",
                "default": "economy"
            }
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [8]:
# And this is included in a list of tools:

tools = [{"type": "function", "function": price_function}]

## Getting OpenAI to use our Tool

There's some fiddly stuff to allow OpenAI "to call our tool"

What we actually do is give the LLM the opportunity to inform us that it wants us to run the tool.

Here's how the new chat function looks:

In [81]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
   
    if response.choices[0].finish_reason=="tool_calls":
        #this means that openai has returned 1 or more tool requests that need to be addressed
        message = response.choices[0].message
        tool_responses = handle_tool_calls(message)
        messages.append(message)
        for tool_response in tool_responses:
            messages.append(tool_response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
    return response.choices[0].message.content

In [83]:
def handle_tool_calls(message):
    
    tool_responses = []
    #iterate through tool calls
    for tool_call in message.tool_calls:
        #json parameters 
        arguments = json.loads(tool_call.function.arguments)
    
        #look up argument
        city = arguments.get('destination_city')
        ticket_class = arguments.get('ticket_class')
    
        #check for empty parameter
        if not ticket_class:
            ticket_class = 'economy'
        #call the tool
        price = get_ticket_price(city, ticket_class)
                                
        #build a response - note role = tool
        response = {
            "role": "tool",
            "content": json.dumps({"destination_city": city,"ticket_class": ticket_class, "price": price}),
            #link response to request using id
            "tool_call_id": tool_call.id
        }
        tool_responses.append(response)
    return tool_responses

In [84]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7884

To create a public link, set `share=True` in `launch()`.


IN CHAT Before Checking for tool calls : response=ChatCompletion(id='chatcmpl-AaPsfsMvh2yG1leVL1fFaM4Dw24Hd', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_ao86rTi80FlHa9vdJ4RlsRhG', function=Function(arguments='{"destination_city": "Rome", "ticket_class": "first"}', name='get_ticket_price'), type='function'), ChatCompletionMessageToolCall(id='call_PTmz2Z17DutHPigINwidprUl', function=Function(arguments='{"destination_city": "Rome", "ticket_class": "business"}', name='get_ticket_price'), type='function')]))], created=1733242657, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier=None, system_fingerprint='fp_0705bf87c0', usage=CompletionUsage(completion_tokens=58, prompt_tokens=171, total_tokens=229, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_token

In [ ]:
how much is a ticket to rome for first class and business class?

